In [ ]:
import pandas as pd
from qgrid import show_grid
import seaborn as sns
from pandas_profiling import ProfileReport
from funpymodeling.exploratory import freq_tbl, status, profiling_num, cat_vars, num_vars
import numpy as np

In [ ]:
# Fuente: https://github.com/rfordatascience/tidytuesday/blob/master/data/2020/2020-01-21/readme.md
data=pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-01-21/spotify_songs.csv")

status(data)

### 2) AED & Storytelling

In [ ]:
data.hist(figsize = (15,20));

In [ ]:
#show_grid(data)
show_grid(data, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 200})

In [ ]:
len(data)

In [ ]:
ProfileReport(data, minimal=True)

### 3) Preparación de datos

In [ ]:
x_data=data.drop(cat_vars(data), axis=1)

In [ ]:
status(x_data)

### 4) Tenemos que normalizar los datos (z-score)

In [ ]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
std_scaler.fit(x_data)
x_data_norm=std_scaler.transform(x_data)

# A ver que hizo!?
pd.DataFrame(x_data_norm).describe()

### 5) Creación del modelo

In [ ]:
from sklearn.cluster import KMeans

cl_model = KMeans(
    n_clusters=3,  # el parámetro importante!
    init='k-means++', # k-means++ acelera la convergencia, respecto de random
    max_iter=100, 
    random_state=0
)

cl_model.fit(x_data_norm)

In [ ]:
pred_cl = cl_model.predict(x_data_norm)

#  Número de cluster: 
pred_cl

### 6) Pequeño paréntesis... ¿Cantidad de clusters?

In [ ]:
cl_model

In [ ]:
from copy import deepcopy

In [ ]:
from yellowbrick.cluster.elbow import kelbow_visualizer

cl_model2=deepcopy(cl_model) # ojo

kelbow_visualizer(cl_model2, x_data_norm, k=(2,9), locate_elbow=False)

Copy-paste de la [doc](https://www.scikit-yb.org/en/latest/api/cluster/elbow.html): _distortion, which computes the sum of squared distances from each point to its assigned center._

### 7) Analizando el modelo final: Gráfico de coordenadas

Hacemos copia de los datos y le agregamos el cluster:

In [ ]:
cluster_var='cluster' # definan nombre de var de clustering
x_data_cl=x_data.copy() # hagan copia de los datos de TR para no perderlos
x_data_cl[cluster_var]=cl_model.predict(x_data_norm) # ojo aca! el predict es de los datos normalizados

In [ ]:
x_data_cl.head()

In [ ]:
from funpymodeling.model_validation import coord_plot

In [ ]:
d_orig, d_transf = coord_plot(x_data_cl, cluster_var)

In [ ]:
d_orig

In [ ]:
d_transf

In [ ]:
status(data)

In [ ]:
# Selección de vars
d2=data[['playlist_genre', 'speechiness','instrumentalness','loudness', 'track_name']]

# Tomamos muestra para que no nos explote el gráfico ;)
d2=d2.sample(n=300)

In [ ]:
# Usamos la maravilla de plotly
import plotly.express as px

df = px.data.iris()
fig = px.scatter_3d(d2, x='speechiness', y='instrumentalness', z='loudness', color='playlist_genre')
fig.show()